In [31]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist

%matplotlib inline
from matplotlib import lines as mlines
import matplotlib.pyplot as plt
from cycler import cycler

import mplhep as hep
plt.style.use([hep.style.CMS])

In [32]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

nfiles_mc = {}
nfiles_mc['2016'] = 113
nfiles_mc['2017'] = 167
nfiles_mc['2018'] = 201

nfiles_data = {}
nfiles_data['2016'] = 101
nfiles_data['2017'] = 97
nfiles_data['2018'] = 133

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [33]:
year = '2018'
outsum = processor.dict_accumulator()

In [34]:
# Load all MC
for n in range(1,nfiles_mc[year]+1):
    print(n)
    filename = 'condor/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates1'].scale(scale_lumi, 'dataset')
outsum['templates2'].scale(scale_lumi, 'dataset')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
File condor/outfiles/2018_58.coffea is missing
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
File condor/outfiles/2018_145.coffea is missing
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


In [35]:
# Load all data
for n in range(1,nfiles_data[year]+1):
    print(n)
    filename = 'condor/outdata/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
# BE CAREFUL NOT TO UNBLIND
# Can you figure out how to blind it at this step???

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
File condor/outdata/2018_69.coffea is missing
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
File condor/outdata/2018_86.coffea is missing
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
File condor/outdata/2018_105.coffea is missing
106
File condor/outdata/2018_106.coffea is missing
107
File condor/outdata/2018_107.coffea is missing
108
File condor/outdata/2018_108.coffea is missing
109
File condor/outdata/2018_109.coffea is missing
110
File condor/outdata/2018_110.coffea is missing
111
File condor/outdata/2018_111.coffea is missing
112
File condor/outdata/2018_112.coffea is missing
113
File condor/outdata/2018_113.coffea is missing
114
File condor/outdata/2018_114.coffea is missing
115
File condor/outdata/2018_115.coffea is missing
116
File condor/outdata/2018_116.coffea

In [36]:
templates = outsum['templates1'].group('dataset', hist.Cat('process', 'Process'), pmap)

/uscms_data/d3/jennetd/hbb-prod-modes/february-2021/inclusive/coffeaenv/lib/python3.7/site-packages/coffea/hist/hist_tools.py:347: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7fc93e869c50>
  warnings.warn("Not all requested indices present in %r" % self, RuntimeWarning)


In [37]:
templates.sum('pt1','msd1').values()

{('ZH', 'signal'): array([11.37746096,  6.7321928 ]),
 ('ZH', 'muoncontrol'): array([0.08861723, 0.        ]),
 ('ZH', 'noselection'): array([87.66226281, 27.10967751]),
 ('WH', 'signal'): array([14.55234195,  8.07146629]),
 ('WH', 'muoncontrol'): array([0.10371754, 0.        ]),
 ('WH', 'noselection'): array([108.07372382,  19.08725118]),
 ('ttH', 'signal'): array([12.06805001,  4.08260177]),
 ('ttH', 'muoncontrol'): array([0.20421442, 0.0606769 ]),
 ('ttH', 'noselection'): array([457.62882561,  76.64599166]),
 ('VBF', 'signal'): array([18.40222327, 27.43118304]),
 ('VBF', 'muoncontrol'): array([0., 0.]),
 ('VBF', 'noselection'): array([76.31343545, 49.56601969]),
 ('ggF', 'signal'): array([41.99369164, 60.05142593]),
 ('ggF', 'muoncontrol'): array([0., 0.]),
 ('ggF', 'noselection'): array([227.3159863 , 118.13174335]),
 ('QCD', 'signal'): array([6262369.854947  ,   84783.73862792]),
 ('QCD', 'muoncontrol'): array([201.47356624,   1.94772686]),
 ('QCD', 'noselection'): array([29871742

In [38]:
templates.sum('msd1','pt1').values()

{('ZH', 'signal'): array([11.37746096,  6.7321928 ]),
 ('ZH', 'muoncontrol'): array([0.08861723, 0.        ]),
 ('ZH', 'noselection'): array([87.66226281, 27.10967751]),
 ('WH', 'signal'): array([14.55234195,  8.07146629]),
 ('WH', 'muoncontrol'): array([0.10371754, 0.        ]),
 ('WH', 'noselection'): array([108.07372382,  19.08725118]),
 ('ttH', 'signal'): array([12.06805001,  4.08260177]),
 ('ttH', 'muoncontrol'): array([0.20421442, 0.0606769 ]),
 ('ttH', 'noselection'): array([457.62882561,  76.64599166]),
 ('VBF', 'signal'): array([18.40222327, 27.43118304]),
 ('VBF', 'muoncontrol'): array([0., 0.]),
 ('VBF', 'noselection'): array([76.31343545, 49.56601969]),
 ('ggF', 'signal'): array([41.99369164, 60.05142593]),
 ('ggF', 'muoncontrol'): array([0., 0.]),
 ('ggF', 'noselection'): array([227.3159863 , 118.13174335]),
 ('QCD', 'signal'): array([6262369.854947  ,   84783.73862792]),
 ('QCD', 'muoncontrol'): array([201.47356624,   1.94772686]),
 ('QCD', 'noselection'): array([29871742

In [39]:
ptbins = [450, 500, 550, 600, 675, 800, 1200]

In [40]:
os.system('rm '+year+'/signalregion.root')
fout = uproot.create(year+'/signalregion.root')
for i,b in enumerate(ptbins[:-1]):
    for p in pmap.keys():  
        h = templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout[p+"_pass_pt"+str(i+1)] = hist.export1d(h)
        h = templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout[p+"_fail_pt"+str(i+1)] = hist.export1d(h)

fout.close()

AttributeError: module 'uproot' has no attribute 'create'

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[0],ptbins[1])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt1.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[1],ptbins[2])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt2.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[2],ptbins[3])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt3.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[3],ptbins[4])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt4.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[4],ptbins[5])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt5.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[5],ptbins[6])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt6.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').sum('pt1').integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-all.png')

In [ ]:
#templates2 = outsum['templates2'].group('dataset', hist.Cat('process', 'Process'), pmap)

os.system('rm '+year+'/muonCR.root')
fout = uproot.create(year+'/muonCR.root')
for p in pmap.keys():  
    h = templates.integrate('region', 'muoncontrol').sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
    fout[p+"_pass"] = hist.export1d(h)
    h = templates.integrate('region', 'muoncontrol').sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
    fout[p+"_fail"] = hist.export1d(h)

fout.close()

In [ ]:
hist.plot1d(templates.integrate('region', 'muoncontrol').sum('pt1').integrate('process','muondata'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-muondata.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'muoncontrol').sum('pt1').integrate('process','ttbar'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-muondata.png')